In [1]:
import pandas as pd 
import numpy as np  
import seaborn as sns  
import pandas_profiling as pp
import matplotlib.pyplot as plt  
from sklearn.preprocessing import LabelEncoder 
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import roc_auc_score,roc_curve
from sklearn.metrics import plot_confusion_matrix
from sklearn import model_selection
from sklearn.ensemble import RandomForestClassifier

from termcolor import colored
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, ExtraTreesClassifier
from sklearn.metrics import classification_report, accuracy_score
from xgboost import XGBRegressor

from lightgbm import LGBMRegressor

import warnings
warnings.filterwarnings("ignore")

from sklearn import set_config
set_config(print_changed_only = False)

sns.set_theme(style = "whitegrid")

print(colored("\n THE REQUIRED LIBRARIES WERE SUCCESFULLY IMPORTED...", color = "green", attrs = ["bold", "dark"]))


 THE REQUIRED LIBRARIES WERE SUCCESFULLY IMPORTED...


In [2]:
heart = pd.read_csv("https://raw.githubusercontent.com/benvictoria17/DataAnalytics/main/dataset/Heart%20Failure%20Prediction/heart_failure_clinical_records_dataset.csv")
df = heart.copy()
df.head(n = 10).style.background_gradient(cmap = "Reds_r").set_properties(**{"font-family" : "Segoe UI"}).hide_index()

age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
75.000000,0,582,0,20,1,265000.000000,1.900000,130,1,0,4,1
55.000000,0,7861,0,38,0,263358.030000,1.100000,136,1,0,6,1
65.000000,0,146,0,20,0,162000.000000,1.300000,129,1,1,7,1
50.000000,1,111,0,20,0,210000.000000,1.900000,137,1,0,7,1
65.000000,1,160,1,20,0,327000.000000,2.700000,116,0,0,8,1
90.000000,1,47,0,40,1,204000.000000,2.100000,132,1,1,8,1
75.000000,1,246,0,15,0,127000.000000,1.200000,137,1,0,10,1
60.000000,1,315,1,60,0,454000.000000,1.100000,131,1,1,10,1
65.000000,0,157,0,65,0,263358.030000,1.500000,138,0,0,10,1
80.000000,1,123,0,35,1,388000.000000,9.400000,133,1,1,10,1


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299 entries, 0 to 298
Data columns (total 13 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   age                       299 non-null    float64
 1   anaemia                   299 non-null    int64  
 2   creatinine_phosphokinase  299 non-null    int64  
 3   diabetes                  299 non-null    int64  
 4   ejection_fraction         299 non-null    int64  
 5   high_blood_pressure       299 non-null    int64  
 6   platelets                 299 non-null    float64
 7   serum_creatinine          299 non-null    float64
 8   serum_sodium              299 non-null    int64  
 9   sex                       299 non-null    int64  
 10  smoking                   299 non-null    int64  
 11  time                      299 non-null    int64  
 12  DEATH_EVENT               299 non-null    int64  
dtypes: float64(3), int64(10)
memory usage: 30.5 KB


In [4]:
df_eda = pd.DataFrame()

df_eda["age"] = df["age"]
df_eda["anaemia"] = np.where(df["anaemia"] < 1, "no", "yes")
df_eda["creatinine_phosphokinase"] = df["creatinine_phosphokinase"]
df_eda["diabetes"] = np.where(df["diabetes"] < 1, "no", "yes")
df_eda["ejection_fraction"] = df["ejection_fraction"]
df_eda["high_blood_pressure"] = np.where(df["high_blood_pressure"] < 1, "no", "yes")
df_eda["platelets"] = df["platelets"]
df_eda["serum_creatinine"] = df["serum_creatinine"]
df_eda["serum_sodium"] = df["serum_sodium"]
df_eda["sex"] = np.where(df["sex"] < 1, "female", "male")
df_eda["smoking"] = np.where(df["smoking"] < 1, "no", "yes")
df_eda["time"] = df["time"]
df_eda["death_event"] = np.where(df["DEATH_EVENT"] < 1, "no", "yes")

df_eda.head().style.background_gradient(cmap = "Reds").set_properties(**{"font-family" : "Segoe UI"}).hide_index()

age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,death_event
75.000000,no,582,no,20,yes,265000.000000,1.900000,130,male,no,4,yes
55.000000,no,7861,no,38,no,263358.030000,1.100000,136,male,no,6,yes
65.000000,no,146,no,20,no,162000.000000,1.300000,129,male,yes,7,yes
50.000000,yes,111,no,20,no,210000.000000,1.900000,137,male,no,7,yes
65.000000,yes,160,yes,20,no,327000.000000,2.700000,116,female,no,8,yes


In [5]:
df_eda.describe().T.style.background_gradient(cmap = "Reds_r").set_properties(**{"font-family" : "Segoe UI"})

,count,mean,std,min,25%,50%,75%,max
age,299.000000,60.833893,11.894809,40.000000,51.000000,60.000000,70.000000,95.000000
creatinine_phosphokinase,299.000000,581.839465,970.287881,23.000000,116.500000,250.000000,582.000000,7861.000000
ejection_fraction,299.000000,38.083612,11.834841,14.000000,30.000000,38.000000,45.000000,80.000000
platelets,299.000000,263358.029264,97804.236869,25100.000000,212500.000000,262000.000000,303500.000000,850000.000000
serum_creatinine,299.000000,1.393880,1.034510,0.500000,0.900000,1.100000,1.400000,9.400000
serum_sodium,299.000000,136.625418,4.412477,113.000000,134.000000,137.000000,140.000000,148.000000
time,299.000000,130.260870,77.614208,4.000000,73.000000,115.000000,203.000000,285.000000


In [6]:
df.isnull().any()

age                         False
anaemia                     False
creatinine_phosphokinase    False
diabetes                    False
ejection_fraction           False
high_blood_pressure         False
platelets                   False
serum_creatinine            False
serum_sodium                False
sex                         False
smoking                     False
time                        False
DEATH_EVENT                 False
dtype: bool

In [7]:
pp.ProfileReport(df_eda)

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

In [8]:

plt.figure(figsize = (18, 10), dpi = 50, edgecolor = "black")

(df_eda["sex"].value_counts(ascending = True, normalize = True)
 .plot.barh(color= ["#FF0000", "#3AFF00"])
 .set_title("Class frequencies of 'sex' variable"));
plt.figure(figsize = (18, 10), dpi = 50, edgecolor = "black")

sns.barplot(x = df["anaemia"].value_counts().index,
            y = df["anaemia"].value_counts(),
            data = df_eda, palette = "Reds",
            saturation = 1).set(title = "The number of anaemia cases (yes - 1, no - 0)");
plt.figure(figsize = (18, 10), dpi = 50)

sns.barplot(x = df_eda["sex"].value_counts().index,
            y = df_eda["death_event"].value_counts(),
            data = df_eda, palette = "Reds", 
            saturation = 1).set(title = "Th e number of death cases per sex");
plt.figure(figsize = (18, 10), dpi = 50)

sns.barplot(y = df_eda["smoking"].value_counts().index,
            x = df_eda["death_event"].value_counts(),
            data = df_eda, palette = "Reds_r", 
            saturation = 1,
            orient = "h").set(title = "The number of death cases and smoking");
sns.catplot(x = "diabetes",
            y = "age",
            kind = "swarm",
            data = df_eda, height = 7, aspect = 1.2, palette = "magma", 
            margin_titles = True).set(title = "Diabet cases and age");
sns.catplot(x = "high_blood_pressure",
            y = "age",
            kind = "strip",
            data = df_eda, height = 7, aspect = 1.2, palette = "magma", 
            margin_titles = True).set(title = "High blood pressure and age");
sns.catplot(x = "anaemia",
            y = "serum_creatinine",
            kind = "box",
            color = '#abc1ed', height = 7, aspect = 1.2, palette = "magma",
            data = df_eda).set(yscale="log").set(title = "Anaemia and serum creatinine");
fig, axes = plt.subplots(1, 3, figsize=(30, 8))
fig.suptitle("")

sns.barplot(ax = axes[0],
            x = "anaemia",
            y = "creatinine_phosphokinase",
            data = df_eda, palette = "gist_earth",
            saturation = 1)
axes[0].set_title("anaemia and creatinine_phosphokinase")

sns.barplot(ax = axes[1],
            x = "anaemia",
            y = "serum_sodium",
            data = df_eda, palette = "Reds",
            saturation = 1)
axes[1].set_title("anaemia and serum sodium")

sns.barplot(ax = axes[2],
            x = "anaemia",
            y = "serum_creatinine",
            data = df_eda, palette = "magma", 
            saturation = 1)
axes[2].set_title("anaemia and serum_creatinine");    
plt.figure(figsize = (18, 10), dpi = 50)

sns.barplot(x = "anaemia", y = "creatinine_phosphokinase", hue = "sex",
            data = df_eda, palette = "Set1", saturation = 1);
fig, axes = plt.subplots(1, 2, figsize=(20, 7))

sns.distplot(ax = axes[0], x = df_eda["age"],
             hist = True,
             bins = 20, 
             kde = True,
             vertical = False, color = "#C44D04").set(title = "The distribution of the values of 'age' variable");

sns.distplot(ax = axes[1], x = df_eda["platelets"],
             hist = True,
             bins = 20,
             kde = True,
             vertical = False, color = "#FA8B47").set(title = "The distribution of the values of 'platelets' variable");
sns.displot(
    data = df_eda,
    x = "age", hue = "sex",
    kind="kde", height=6,
    multiple="fill", clip=(0, None),
    color = "#FE6203",
).set(title = "density of the classes of 'sex' variable by age");


sns.displot(
    data = df_eda,
    x = "age", hue = "death_event",
    kind = "kde", height=6,
    multiple="fill", clip=(0, None),
    palette="ch:rot=-.25,hue=1,light=.75",
).set(title = "density of the classes of 'death_event' variable by age");

In [9]:
fig, axes = plt.subplots(1, 2, figsize=(20, 10))


sns.boxplot(ax = axes[0], x = "sex", y = "serum_sodium", data = df_eda, width = 0.7,
            orient = "v", fliersize = 3, linewidth = 1);
axes[0].set_title("relationship between 'sex' and 'serum-sodium' variables");

sns.boxplot(ax = axes[1], x = "sex", y = "platelets", data = df_eda, width = 0.7,
            orient = "v", fliersize = 3, linewidth = 1);
axes[1].set_title("relationship between 'sex' and 'plateles' variables");
sns.pairplot(df_eda, hue = "sex", diag_kind = "hist", corner = True);
fig, axes = plt.subplots(2, 2, figsize = (20, 15))
axes = axes.flatten()

sns.scatterplot(ax = axes[0],
                x = "age",
                y = "serum_creatinine",  palette = "Reds_r",
                data = df_eda).set(title = 'Relationship between age and serum creatinine level');

sns.scatterplot(ax = axes[1],
                x = "age",
                y = "platelets", palette = "Reds_r",
                data = df_eda).set(title = 'Relationship between age and platelets');

sns.scatterplot(ax = axes[2],
                x = "age",
                y = "serum_sodium", palette = "Reds_r",
                data = df_eda).set(title = 'Relationship between age and serum_sodium');

sns.scatterplot(ax = axes[3],
                x = "age",
                y = "creatinine_phosphokinase",  palette = "Reds_r",
                data = df_eda).set(title = 'Relationship between age and creatinine_phosphokinase');
fig, axes = plt.subplots(2, 2, figsize=(20, 14))
axes = axes.flatten()

sns.scatterplot(ax = axes[0],
                x = "sex",
                y = "serum_creatinine",
                data = df_eda).set(title = 'Relationship between sex and serum creatinine level');

sns.scatterplot(ax = axes[1],
                x = "sex",
                y = "platelets",
                data = df_eda).set(title = 'Relationship between sex and platelets');

sns.scatterplot(ax = axes[2],
                x = "sex",
                y = "serum_sodium",
                data = df_eda).set(title = 'Relationship between sex and serum_sodium');

sns.scatterplot(ax = axes[3],
                x = "sex",
                y = "creatinine_phosphokinase",
                data = df_eda).set(title = 'Relationship between sex and creatinine_phosphokinase');
fig, axes = plt.subplots(2, 2, figsize=(20, 14))
axes = axes.flatten()

sns.scatterplot(ax = axes[0],
                x = "diabetes",
                y = "serum_creatinine",
                data = df_eda).set(title = 'Relationship between diabetes and serum creatinine level');

sns.scatterplot(ax = axes[1],
                x = "diabetes",
                y = "platelets",
                data = df_eda).set(title = 'Relationship between diabetes and platelets');

sns.scatterplot(ax = axes[2],
                x = "diabetes",
                y = "serum_sodium",
                data = df_eda).set(title = 'Relationship between diabetes and serum_sodium');

sns.scatterplot(ax = axes[3],
                x = "diabetes",
                y = "creatinine_phosphokinase",
                data = df_eda).set(title = 'Relationship between diabetes and creatinine_phosphokinase');
df_eda["sex_0_1"] = np.where(df_eda["sex"] == "male", 1, 0)

fig, axes = plt.subplots(figsize = (12, 7))

sns.set_theme(style = "darkgrid")

sns.scatterplot(data = df_eda, x = "age", y = "platelets", hue = "sex_0_1",
    palette = "Reds_r", size = "sex_0_1", sizes = (50, 200), hue_norm = (2, 8),
    legend="auto").set(title = "relationship between 'age' and 'platelets' variables by 'sex' variable");
sns.set_theme(style = "darkgrid")
g = sns.JointGrid(data = df_eda, x = "age", y = "serum_sodium", space = 0)

g.plot_joint(sns.kdeplot, fill = True, thresh = 0, cmap = "Reds_r")
g.plot_marginals(sns.histplot, color = "#FE6203", alpha = 1, bins = 25);
sns.set_theme(style = "darkgrid")
g = sns.JointGrid(data = df_eda, x = "creatinine_phosphokinase", y = "serum_creatinine", space = 0)

g.plot_joint(sns.kdeplot, fill = True, thresh = 0, cmap = "Reds")
g.plot_marginals(sns.histplot, color = "#FE6203", alpha = 1, bins = 25);
mask = np.zeros_like(df_eda.corr())
triangle_indices = np.triu_indices_from(mask)
mask[triangle_indices] = True


plt.figure(figsize = (18, 10))
sns.heatmap(df_eda.corr(), mask = mask, annot = True, annot_kws = {"size": 12}, robust = True, linewidths = 2.5)
sns.set_style(style = "darkgrid")
plt.xticks(fontsize = 12)
plt.yticks(fontsize = 12)
plt.show()
sns.lmplot(x = "age", y = "platelets", hue = "sex", 
           height = 7, aspect = 1.2, markers = ["*", "s"],
           data = df_eda).set(title = 'relationship between "age" and "platelets" variables, by "sex" variable');
fig, ([ax0, ax1], [ax2, ax3]) = plt.subplots(nrows = 2,
                               ncols = 2,
                               sharey = True,
                               figsize = (25, 18))

hb = ax0.hexbin(df_eda["age"],
                df_eda["serum_sodium"],
                gridsize = 30,
                cmap = 'inferno')
ax0.set_title("Hexagon binning")
cb = fig.colorbar(hb, ax = ax0, label = 'counts')


hb = ax1.hexbin(df_eda["age"],
                df_eda["serum_sodium"],
                gridsize = 30,
                bins = 'log',
                cmap = 'Reds')
ax1.set_title("With a log color scale")
cb = fig.colorbar(hb, ax = ax1, label = 'log10(N)')

hb = ax2.hexbin(df_eda["platelets"],
                df_eda["serum_sodium"],
                gridsize = 30,
                cmap = 'Reds_r')
ax2.set_title("Hexagon binning")
cb = fig.colorbar(hb, ax = ax2, label = 'counts')

hb = ax3.hexbin(df_eda["platelets"],
                df_eda["serum_sodium"],
                gridsize = 30,
                bins = 'log',
                cmap = 'inferno')
ax3.set_title("With a log color scale")
cb = fig.colorbar(hb, ax = ax3, label = 'log10(N)')


plt.show()
plt.style.use('dark_background')

x = df_eda["age"]
y = df_eda["platelets"]

fig, ax = plt.subplots(figsize = (16, 8))

ax.hist2d(x, y, density = True)

plt.show()
plt.style.use('ggplot')

x = df_eda["creatinine_phosphokinase"]
y = df_eda["serum_sodium"]

fig, ax = plt.subplots(figsize = (14, 10))

ax.hist2d(x, y, density = True)

plt.show()
plt.style.use('seaborn-paper')

pd.crosstab(df_eda['sex'], df_eda['death_event'],normalize = True).plot(kind = "pie", subplots = True, stacked = True );

pd.crosstab(df_eda['diabetes'], df_eda['death_event'], normalize = True).plot(kind = "line", stacked = True);

pd.crosstab(df_eda['anaemia'], df_eda['death_event'], normalize = True).plot(kind = "area", stacked = True);

pd.crosstab(df_eda['high_blood_pressure'], df_eda['death_event'], normalize = True).plot(kind = "bar", stacked = True);

pd.crosstab(df_eda['smoking'], df_eda['death_event'], normalize = True).plot(kind = "kde", stacked = True);
plt.figure(figsize = (18, 10), dpi = 50)
plt.style.use('ggplot')
sns.lineplot(data = df_eda, x = "age", y = "time", hue = "sex" );

In [11]:
!pip install autoviz

In [13]:
plt.figure(figsize = (10, 5))
from autoviz.AutoViz_Class import AutoViz_Class
AV = AutoViz_Class()
df_av = AV.AutoViz('https://raw.githubusercontent.com/benvictoria17/DataAnalytics/main/dataset/Heart%20Failure%20Prediction/heart_failure_clinical_records_dataset.csv')
plt.show()

Shape of your Data Set loaded: (299, 13)
#######################################################################################
######################## C L A S S I F Y I N G  V A R I A B L E S  ####################
#######################################################################################
Classifying variables in data set...
    13 Predictors classified...
        No variables removed since no ID or low-information variables found in data set
Number of All Scatter Plots = 6
All Plots done
Time to run AutoViz = 4 seconds 

 ###################### AUTO VISUALIZATION Completed ########################


In [14]:
df.head().style.background_gradient(cmap = "Reds_r")

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.000000,0,582,0,20,1,265000.000000,1.900000,130,1,0,4,1
1,55.000000,0,7861,0,38,0,263358.030000,1.100000,136,1,0,6,1
2,65.000000,0,146,0,20,0,162000.000000,1.300000,129,1,1,7,1
3,50.000000,1,111,0,20,0,210000.000000,1.900000,137,1,0,7,1
4,65.000000,1,160,1,20,0,327000.000000,2.700000,116,0,0,8,1


In [15]:
y = df["DEATH_EVENT"]

x = df.drop("DEATH_EVENT", axis = 1)

print(colored("\n DEPENDENT AND INDEPENDENT VARIABLES WERE SUCCESFULLY SELECTED...", color = "green", attrs = ["bold", "dark"]))


 DEPENDENT AND INDEPENDENT VARIABLES WERE SUCCESFULLY SELECTED...


In [17]:
x_train, x_test, y_train, y_test = train_test_split(x, y,
                                                    test_size = 0.20,
                                                    shuffle = True,
                                                    random_state = 1)

print(colored("\n THE DATASET WERE SUCCESFULLY SPLITTED - test = 20%, train = 80%...", color = "green", attrs = ["bold", "dark"]))


 THE DATASET WERE SUCCESFULLY SPLITTED - test = 20%, train = 80%...


In [18]:
scaler = StandardScaler()
scaler.fit(x_train)
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)

print(colored("\n x_train AND x_test SETS WERE SUCCESFULLY STANDARIZED", color = "green", attrs = ["bold", "dark"]))


 x_train AND x_test SETS WERE SUCCESFULLY STANDARIZED


In [19]:
LRclassifier = LogisticRegression(max_iter=1000, random_state=1, solver='liblinear', penalty='l1')
LRclassifier.fit(x_train, y_train)

y_pred_LR = LRclassifier.predict(x_test)
LRAcc = accuracy_score(y_pred_LR, y_test)
print('.:. Logistic Regression Accuracy:'+'\033[1m {:.2f}%'.format(LRAcc*100)+' .:.')

.:. Logistic Regression Accuracy: 86.67% .:.


In [20]:
KNNClassifier = KNeighborsClassifier(n_neighbors=3)
KNNClassifier.fit(x_train, y_train)

y_pred_KNN = KNNClassifier.predict(x_test)
KNNAcc = accuracy_score(y_pred_KNN, y_test)
print('.:. K-Nearest Neighbour Accuracy:'+'\033[1m {:.2f}%'.format(KNNAcc*100)+' .:.')

.:. K-Nearest Neighbour Accuracy: 80.00% .:.


In [21]:
SVMclassifier = SVC(kernel='linear', max_iter=1000, C=10, probability=True)
SVMclassifier.fit(x_train, y_train)

y_pred_SVM = SVMclassifier.predict(x_test)
SVMAcc = accuracy_score(y_pred_SVM, y_test)
print('.:. Support Vector Machine Accuracy:'+'\033[1m {:.2f}%'.format(SVMAcc*100)+' .:.')

.:. Support Vector Machine Accuracy: 73.33% .:.


In [22]:
GNBclassifier = GaussianNB(var_smoothing=0.1)
GNBclassifier.fit(x_train, y_train)

y_pred_GNB = GNBclassifier.predict(x_test)
GNBAcc = accuracy_score(y_pred_GNB, y_test)
print('.:. Gaussian Naive Bayes Accuracy:'+'\033[1m {:.2f}%'.format(GNBAcc*100)+' .:.')

.:. Gaussian Naive Bayes Accuracy: 81.67% .:.


In [23]:
DTCclassifier = DecisionTreeClassifier(max_depth=3, min_samples_leaf=5, criterion='entropy', min_samples_split=5,
                                       splitter='random', random_state=1)
DTCclassifier.fit(x_train, y_train)
y_pred_DTC = DTCclassifier.predict(x_test)
DTCAcc = accuracy_score(y_pred_DTC, y_test)
print('.:. Decision Tree Accuracy:'+'\033[1m {:.2f}%'.format(DTCAcc*100)+' .:.')

.:. Decision Tree Accuracy: 75.00% .:.


In [24]:
RFclassifier = RandomForestClassifier(n_estimators=1000, random_state=1, max_leaf_nodes=20, min_samples_split=15)

RFclassifier.fit(x_train, y_train)
y_pred_RF = RFclassifier.predict(x_test)
RFAcc = accuracy_score(y_pred_RF, y_test)
print('.:. Random Forest Accuracy:'+'\033[1m {:.2f}%'.format(RFAcc*100)+' .:.')

.:. Random Forest Accuracy: 90.00% .:.


In [25]:
GBclassifier = GradientBoostingClassifier(random_state=1, n_estimators=100, max_leaf_nodes=3, loss='exponential', 
                                          min_samples_leaf=20)

GBclassifier.fit(x_train, y_train)
y_pred_GB = GBclassifier.predict(x_test)
GBAcc = accuracy_score(y_pred_GB, y_test)
print('.:. Gradient Boosting Accuracy:'+'\033[1m {:.2f}%'.format(GBAcc*100)+' .:.')

.:. Gradient Boosting Accuracy: 85.00% .:.


In [26]:
compare = pd.DataFrame({'Model': ['Logistic Regression', 'K-Nearest Neighbour', 'Support Vector Machine', 
                                  'Gaussian Naive Bayes', 'Decision Tree', 'Random Forest', 'Gradient Boosting'
                                  ], 
                        'Accuracy': [LRAcc*100, KNNAcc*100, SVMAcc*100, GNBAcc*100, DTCAcc*100, RFAcc*100, GBAcc*100, 
                                     ]})

compare.sort_values(by='Accuracy', ascending=False).style.background_gradient(cmap='PuRd').hide_index().set_properties(**{'font-family': 'Segoe UI'})

Model,Accuracy
Random Forest,90.000000
Logistic Regression,86.666667
Gradient Boosting,85.000000
Gaussian Naive Bayes,81.666667
K-Nearest Neighbour,80.000000
Decision Tree,75.000000
Support Vector Machine,73.333333
